## <center> Isosurface extracted by Meshing.jl</center>

Given a 3d array representing the values of a scalar field, we illustrate how `Meshing.jl` extracts the brain surface  mesh, as an isosurface, derived via Marching Cube algorithm. The actual implementation of marching cube has the drawback that a vertex is not unique in the vector of mesh vertices, leading to big meshes [https://github.com/JuliaGeometry/Meshing.jl/issues/83](https://github.com/JuliaGeometry/Meshing.jl/issues/83).

Our dataset is stored in an npy file that contains the so called MNI152 (MNI - Montreal Neurological Institute) averaged brain template (it is an average brain taken from MRI's of 152 healthy individuals). This file was derived from a NIfTI image. 

In [ ]:
using Meshing
using StaticArrays
using PlotlyJS
using NPZ
import Rotations.RotY
include("src/plotlyju.jl");

In [ ]:
brain_vol = npzread("data/MNI152.npy") 
verts, triangles =  Meshing.isosurface(brain_vol, MarchingCubes(iso=1.25, reduceverts=true, eps=1e-05));
print(length(verts))

uni_verts = unique(i -> verts[i], 1:length(verts))
print(length(uni_verts))

matr_tri = reshape(reinterpret(Int64, triangles), (3, size(triangles,1)))';
matr_tri = matr_tri .- 1; #make indices starting from 0
matr_verts = reshape(reinterpret(Float64, verts), (3, size(verts,1)));
rmatr_verts = (RotY(-pi/9) * matr_verts)'; #data rotation to ilustrate more from the brain, as static image

pl_brain=[[0, "rgb(190,190,190)"], [1, "rgb(190,190,190)"]]
tr = mesh3d(x=rmatr_verts[:, 1],
            y=rmatr_verts[:, 2],
            z=rmatr_verts[:, 3],
            i=matr_tri[:, 1],
            j=matr_tri[:, 2],
            k=matr_tri[:, 3],
            intensity=matr_verts[:, 3],
            colorscale=pl_brain,
            showscale=false,
            flatshading=true,
            lighting=attr(ambient=0.65,
                          diffuse=0.5,
                          fresnel=0.25,        
                          specular=0.25,
                          roughness=0.25,
                          facenormalsepsilon=0,
                          vertexnormalsepsilon=0),
            lightposition=attr(x=100,
                               y=100,
                               z=10) )

layout = Layout( 
         width=500,
         height=500,
         paper_bgcolor="rgb(10, 10,10)",
         margin=attr(t=2, r=2, b=2, l=2),
         scene=attr(xaxis_visible=false,
                    yaxis_visible=false, 
                    zaxis_visible=false, 
                    aspectmode="data",
                    camera_eye=attr(x=1.45, y=1.45, z=0.2)
                    ))
        

pl=plot(tr, layout, style=plotlyju)

![iso-brain](images/iso-brain.png)

A slice in this brain is defined in the notebook `16-Brain-slice.ipynb`